<a href="https://colab.research.google.com/github/TimHBSWFL/UCSD-ML-Capstone/blob/main/hyperparameter_tuning_business_attributes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 2.2 MB/s eta 0:00:00


In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import optuna
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error, r2_score
from xgboost import XGBRegressor
import xgboost as xgb

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
filename = "FL_Restaurants_Business Attributes_Edited" + ".csv"
directory = '/content/drive/My Drive/Capstone Data Collection/'

path = directory + filename

chunk_iterator = pd.read_csv(path, chunksize=10000)

chunks = []

for chunk in chunk_iterator:
  chunks.append(chunk)

df = pd.concat(chunks, ignore_index=True)
df.shape

(8723, 166)

In [14]:
df2 = df.copy()

XGBoost Regressor Hyperparameter Tuning

In [15]:
X = df.drop(columns=['stars'])
y = df['stars']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

def objective(trial):
    params = {
      "n_estimators": trial.suggest_int("n_estimators", 50, 500),
      "max_depth": trial.suggest_int("max_depth", 3, 15),
      "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
      "subsample": trial.suggest_float("subsample", 0.6, 1.0),
      "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
      "random_state": 42,
      "objective": "reg:squarederror"
  }

    model = XGBRegressor(**params)
    model.fit(X_train_scaled, y_train)

    y_pred = model.predict(X_test_scaled)
    y_pred_rounded = np.round(y_pred * 2) / 2

    mse = mean_squared_error(y_test, y_pred_rounded)
    return mse


study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

print("Best Parameters:", study.best_params)
print("Best MSE:", study.best_value)


[I 2024-11-29 21:51:02,520] A new study created in memory with name: no-name-3118025a-6839-4742-ba69-61c7bb841c9a
[I 2024-11-29 21:51:08,767] Trial 0 finished with value: 0.014326647564469915 and parameters: {'n_estimators': 248, 'max_depth': 9, 'learning_rate': 0.11528045403735948, 'subsample': 0.8906725354004891, 'colsample_bytree': 0.6753077670087334}. Best is trial 0 with value: 0.014326647564469915.
[I 2024-11-29 21:51:09,701] Trial 1 finished with value: 0.012320916905444125 and parameters: {'n_estimators': 364, 'max_depth': 4, 'learning_rate': 0.08078950017227297, 'subsample': 0.7501871032303858, 'colsample_bytree': 0.7001288918378585}. Best is trial 1 with value: 0.012320916905444125.
[I 2024-11-29 21:51:10,580] Trial 2 finished with value: 0.01489971346704871 and parameters: {'n_estimators': 122, 'max_depth': 9, 'learning_rate': 0.2565082822291618, 'subsample': 0.6730897656057443, 'colsample_bytree': 0.7156584836843521}. Best is trial 1 with value: 0.012320916905444125.
[I 202

Best Parameters: {'n_estimators': 200, 'max_depth': 3, 'learning_rate': 0.15228155915476052, 'subsample': 0.7495728707485861, 'colsample_bytree': 0.7458843432111124}
Best MSE: 0.012034383954154728


In [16]:

best_params = study.best_params

print("Best Parameters from Optuna:", best_params)


X = df.drop(columns=['stars'])
y = df['stars']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


rg_model = XGBRegressor(
    objective='reg:squarederror',
    **best_params
)

rg_model.fit(X_train_scaled, y_train)

y_pred = rg_model.predict(X_test_scaled)

y_pred_rounded = np.round(y_pred * 2) / 2

mse = mean_squared_error(y_test, y_pred_rounded)
r2 = r2_score(y_test, y_pred_rounded)

print("Original Predictions:", y_pred)
print("Rounded Predictions:", y_pred_rounded)
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

accuracy = np.mean(y_pred_rounded == y_test.values)
print(f"Accuracy: {accuracy * 100:.2f}%")


Best Parameters from Optuna: {'n_estimators': 200, 'max_depth': 3, 'learning_rate': 0.15228155915476052, 'subsample': 0.7495728707485861, 'colsample_bytree': 0.7458843432111124}
Original Predictions: [1.9054453 4.77953   4.5043216 ... 2.5497093 4.020626  4.4218073]
Rounded Predictions: [2.  5.  4.5 ... 2.5 4.  4.5]
Mean Squared Error: 0.012320916905444125
R^2 Score: 0.9825460302366446
Accuracy: 95.07%


In [17]:
from joblib import dump

model_directory = '/content/drive/My Drive/Capstone Data Collection/models/xgb_regressor_model.joblib'
scaler_path = '/content/drive/My Drive/Capstone Data Collection/models/scaler.joblib'

dump(rg_model, model_directory)
dump(scaler, scaler_path)

print(f"Model saved successfully to {model_directory}")
print(f"Scaler saved successfully to {scaler_path}")

Model saved successfully to /content/drive/My Drive/Capstone Data Collection/models/xgb_regressor_model.joblib
Scaler saved successfully to /content/drive/My Drive/Capstone Data Collection/models/scaler.joblib


XGBoost Classifier Hyperparameter Tuning

In [18]:
rating_mapping = {1: 0, 1.5: 1, 2: 2, 2.5: 3, 3: 4, 3.5: 5, 4: 6, 4.5: 7, 5: 8}
df2['rating_class'] = df2['stars'].map(rating_mapping)

X = df2.drop(columns=['stars', 'rating_class'])
y = df2['rating_class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


clf_scaler = StandardScaler()

X_train_scaled = clf_scaler.fit_transform(X_train)
X_test_scaled = clf_scaler.transform(X_test)

In [19]:
def objective(trial):
    params = {
        "objective": "multi:softmax",
        "num_class": 9,
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "eval_metric": "mlogloss",
    }


    clf_model = XGBClassifier(**params)

    clf_model.fit(X_train_scaled, y_train)

    y_pred = clf_model.predict(X_test_scaled)


    accuracy = accuracy_score(y_test, y_pred)

    return accuracy


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)


best_params = study.best_params
print("Best Parameters from Optuna:", best_params)


[I 2024-11-29 21:56:37,662] A new study created in memory with name: no-name-c0384299-d14a-4619-8e21-dbb8abcd3673
[I 2024-11-29 21:56:41,023] Trial 0 finished with value: 0.9415472779369628 and parameters: {'max_depth': 4, 'learning_rate': 0.281090812555558, 'n_estimators': 147, 'subsample': 0.9549943160725486, 'colsample_bytree': 0.7088444359853587}. Best is trial 0 with value: 0.9415472779369628.
[I 2024-11-29 21:56:48,222] Trial 1 finished with value: 0.9484240687679083 and parameters: {'max_depth': 12, 'learning_rate': 0.029583340721666122, 'n_estimators': 107, 'subsample': 0.8680765139547704, 'colsample_bytree': 0.6382051924952287}. Best is trial 1 with value: 0.9484240687679083.
[I 2024-11-29 21:56:59,509] Trial 2 finished with value: 0.9461318051575931 and parameters: {'max_depth': 3, 'learning_rate': 0.08519848946590901, 'n_estimators': 418, 'subsample': 0.8446366486040602, 'colsample_bytree': 0.7335733973846059}. Best is trial 1 with value: 0.9484240687679083.
[I 2024-11-29 21

Best Parameters from Optuna: {'max_depth': 4, 'learning_rate': 0.012662505336105573, 'n_estimators': 400, 'subsample': 0.6680223808878537, 'colsample_bytree': 0.8821970262420972}


In [20]:
best_model = XGBClassifier(**best_params)


best_model.fit(X_train_scaled, y_train)

y_pred = best_model.predict(X_test_scaled)


accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of the optimized model: {accuracy * 100:.2f}%")

Accuracy of the optimized model: 95.13%


In [21]:

model_directory = '/content/drive/My Drive/Capstone Data Collection/models/xgb_classifier_model.joblib'
scaler_path = '/content/drive/My Drive/Capstone Data Collection/models/clf_scaler.joblib'

dump(best_model, model_directory)
dump(clf_scaler, scaler_path)

print(f"Model saved successfully to {model_directory}")
print(f"Scaler saved successfully to {scaler_path}")


Model saved successfully to /content/drive/My Drive/Capstone Data Collection/models/xgb_classifier_model.joblib
Scaler saved successfully to /content/drive/My Drive/Capstone Data Collection/models/clf_scaler.joblib
